In [1]:
import os
os.chdir('/home/ubuntu/code/pump_post_midterm/pump')

In [2]:
# # Get test_q_keys
# import random
# import pandas as pd
# from src.utils import get_file_from_s3

# survey_name = "American_Trends_Panel_W34"
# random.seed(42)
# survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))
# q_keys = list(survey_df['key'])
# test_q_keys = random.choices(q_keys, k=5)

# print(test_q_keys)

In [3]:
import random
random.seed(42)

test_q_keys_mapping = {
    "American_Trends_Panel_W26": ['GUNRESPNOKIDSB_W26', 'WORLDDANGER_W26', 'GUNIDENTITY_W26', 'REASONGUNC_W26', 'GUNRESPKIDSC_W26'],
    "American_Trends_Panel_W27": ['CAREGIV3A_W27', 'PREDICTA_W27', 'WORK3C_W27', 'CARS10D_W27', 'CAREGIV7_W27'],
    "American_Trends_Panel_W29": ['TRAITSD_W29', 'NOWSMK_NHIS_W29', 'HELPHURTE_W29', 'HELPHURTA_W29', 'GIRLSF2A_W29'],
    "American_Trends_Panel_W32": ['NEIGHINTERB_W32', 'SATLIFEC_W32', 'SUCCESSIMPA_W32', 'COMMIMPE_W32', 'GROWUPUSR_W32'],
    "American_Trends_Panel_W34": ['MED4A_W34', 'SCI2A_W34', 'FUD22_W34', 'EAT3H_W34', 'MED6D_W34']
}


# Hyper parameters

In [4]:

survey_names = [
    "American_Trends_Panel_W26",
    "American_Trends_Panel_W27",
    "American_Trends_Panel_W29",
    "American_Trends_Panel_W32",
    "American_Trends_Panel_W34"
]

def skip_exp(filename):
    # if ('0828' not in filename or 'sonnetpred' not in filename) and 'prompt2' not in filename: return True
    if 'prompt2' not in filename: return True
    return False

def get_exp(filename):
    if filename[-5:] == '.json': filename = filename[:-5]
    exp_elements = [_ for _ in filename.split('_') if _ in ['history', 'demo', 'persona', 'prompt2']]
    final = '_'.join(exp_elements)
    # print(filename, '->', final)
    return final

# Main

In [5]:
import json
import os
import pandas as pd
import numpy as np
import pandas as pd
import random

from collections import Counter
from src.utils import get_file_from_s3

def get_full_results(survey_name):
    res = {}
    for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):
        if skip_exp(filename): continue
        exp = get_exp(filename)
        # exp = filename
        if exp not in res:
            res[exp] = []
            
        with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
            data = json.load(f)
        res[exp] += [f"={sum([_['is_correct'] for _ in data])}/{len(data)}", sum([_['is_correct'] for _ in data])/len(data)]


    df = pd.DataFrame(res)
    df = df[sorted(df.columns)]
    df = df.transpose()

    acc_df = df
    acc_df = acc_df.rename(columns={0: "acc_cnt", 1: "acc"})
    acc_df.sort_values(by='acc', ascending=False)


    test_q_keys = test_q_keys_mapping[survey_name]
    survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))
    sub_df = survey_df[survey_df['key'].isin(test_q_keys)]

    wd_question_mappings = {row['key']: dict(zip(eval(row['references'].lower()), eval(row['option_ordinal']))) for _, row in sub_df.iterrows()}
    wd_question_mappings

    records = {}
    for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):
        if skip_exp(filename): continue

        with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
            data = json.load(f)
        
        exp = get_exp(filename)

        res = []
        pred_missing_cnt = 0
        gold_missing_cnt = 0
        for entry in data:
            q_idx = entry['q_idx']
            q_key = test_q_keys[q_idx]
            pos_mapping = wd_question_mappings[q_key]
            prediction = entry['prediction'].lower()
            gold_answer = entry['gold_answer'].lower()
            if gold_answer not in pos_mapping:
                gold_missing_cnt += 1
                continue
            if prediction not in pos_mapping:
                pred_missing_cnt += 1
                if prediction not in entry['references']:
                    refs = '/'.join(entry['references'].split('\n'))
                    print(f"Prediction: |{prediction}|, Gold: |{gold_answer}|, References: |{refs}|")
                continue
                prediction = random.choice(list(pos_mapping.keys()))
            pred_pos = pos_mapping[prediction]
            gold_pos = pos_mapping[gold_answer]
            res.append(abs(pred_pos-gold_pos))
        records[exp] = [Counter(res), np.mean(res), pred_missing_cnt, gold_missing_cnt]

    q_wd_df = pd.DataFrame(records)
    q_wd_df = q_wd_df[sorted(q_wd_df.columns)].transpose()
    q_wd_df = q_wd_df.rename(columns={0: "q_1wd_cnt", 1: "q_1wd", 2: "q_1wd_pmc", 3: "q_1wd_gmc"})
    q_wd_df

    merged_df = pd.merge(acc_df, q_wd_df, left_index=True, right_index=True)
    # merged_df = pd.merge(merged_df, s_wd_df, left_index=True, right_index=True)
    # merged_df = merged_df[['wd_avg', 'wd_std', 'acc_avg', 'acc_std']]

    merged_df = merged_df.sort_values(by='q_1wd', ascending=True)
    merged_df = merged_df.sort_values(by='acc', ascending=False)

    # merged_df.sort_values(by='acc_avg', ascending=False)
    # merged_df.sort_values(by='wd_avg', ascending=True)
    # merged_df = merged_df.sort_values(by='s_1wd', ascending=True)

    merged_df['acc_cnt'] = merged_df['acc_cnt'].apply(lambda x: '\''+str(x))
    merged_df

    return merged_df

# Results

In [6]:
from IPython.display import display

for survey_name in survey_names:
    print(survey_name)
    merged = get_full_results(survey_name)
    display(merged)
    print('\n\n\n')

American_Trends_Panel_W26


,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
prompt2_history_demo,'=666/1090,0.611009,"{0.0: 666, 1.0: 258, 0.5: 82, 2.0: 64, 3.0: 15}",0.435023,0,5
prompt2_history,'=660/1090,0.605505,"{0.0: 660, 1.0: 264, 0.5: 82, 2.0: 68, 3.0: 11}",0.436866,0,5
prompt2_demo,'=618/1090,0.566972,"{1.0: 306, 0.0: 618, 0.5: 82, 2.0: 75, 3.0: 4}",0.469124,0,5
prompt2_history_persona,'=613/1090,0.562385,"{0.0: 613, 1.0: 332, 0.5: 86, 2.0: 45, 3.0: 9}",0.453456,0,5
prompt2_history_demo_persona,'=605/1090,0.555046,"{0.0: 605, 1.0: 344, 0.5: 86, 2.0: 45, 3.0: 5}",0.453456,0,5
prompt2_persona,'=485/1090,0.444954,"{2.0: 86, 0.0: 485, 1.0: 419, 0.5: 87, 3.0: 8}",0.606912,0,5
prompt2_demo_persona,'=473/1090,0.433945,"{1.0: 450, 0.0: 473, 0.5: 98, 2.0: 61, 3.0: 3}",0.580645,0,5






American_Trends_Panel_W27
Prediction: |very worried|, Gold: |somewhat enthusiastic|, References: |Very enthusiastic/Somewhat enthusiastic/Not too enthusiastic/Not at all enthusiastic/Refused|
Prediction: |somewhat worried|, Gold: |not too enthusiastic|, References: |Very enthusiastic/Somewhat enthusiastic/Not too enthusiastic/Not at all enthusiastic/Refused|
Prediction: |somewhat worried|, Gold: |somewhat enthusiastic|, References: |Very enthusiastic/Somewhat enthusiastic/Not too enthusiastic/Not at all enthusiastic/Refused|
Prediction: |very worried|, Gold: |not too enthusiastic|, References: |Very enthusiastic/Somewhat enthusiastic/Not too enthusiastic/Not at all enthusiastic/Refused|
Prediction: |very worried|, Gold: |somewhat enthusiastic|, References: |Very enthusiastic/Somewhat enthusiastic/Not too enthusiastic/Not at all enthusiastic/Refused|
Prediction: |very worried|, Gold: |not too enthusiastic|, References: |Very enthusiastic/Somewhat enthusiastic/Not too enthusiastic/No

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
prompt2_history_demo,'=780/1510,0.516556,"{1.0: 610, 0.0: 780, 0.5: 103, 2.0: 11, 3.0: 1}",0.456146,0,5
prompt2_history,'=774/1510,0.512583,"{1.0: 607, 0.0: 774, 0.5: 106, 2.0: 15, 3.0: 1}",0.461078,2,5
prompt2_history_persona,'=756/1510,0.500662,"{0.0: 756, 0.5: 119, 1.0: 607, 2.0: 16, 3.0: 3}",0.471352,4,5
prompt2_history_demo_persona,'=753/1510,0.498675,"{1.0: 610, 0.0: 753, 0.5: 113, 2.0: 20, 3.0: 6}",0.482357,3,5
prompt2_demo,'=605/1510,0.400662,"{2.0: 61, 0.0: 605, 1.0: 731, 0.5: 106, 3.0: 2}",0.60598,0,5
prompt2_demo_persona,'=600/1510,0.397351,"{1.0: 704, 0.0: 600, 0.5: 146, 2.0: 53, 3.0: 2}",0.590698,0,5
prompt2_persona,'=562/1510,0.372185,"{1.0: 708, 0.0: 562, 0.5: 181, 2.0: 52, 3.0: 2}",0.603654,0,5






American_Trends_Panel_W29


,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
prompt2_history,'=1011/1600,0.631875,"{0.0: 1010, 1.0: 249, 0.5: 268, 2.0: 58}",0.314826,0,15
prompt2_history_demo,'=983/1600,0.614375,"{0.0: 983, 1.0: 264, 0.5: 283, 2.0: 55}",0.325237,0,15
prompt2_history_demo_persona,'=937/1600,0.585625,"{0.0: 937, 1.0: 290, 0.5: 295, 2.0: 63}",0.355521,0,15
prompt2_history_persona,'=931/1600,0.581875,"{0.0: 931, 1.0: 286, 0.5: 294, 2.0: 74}",0.366562,0,15
prompt2_demo,'=885/1600,0.553125,"{0.0: 885, 1.0: 267, 0.5: 373, 2.0: 59, 3.0: 1}",0.362461,0,15
prompt2_demo_persona,'=848/1600,0.53,"{0.0: 848, 1.0: 297, 0.5: 344, 2.0: 96}",0.417035,0,15
prompt2_persona,'=773/1600,0.483125,"{0.0: 773, 1.0: 297, 0.5: 404, 2.0: 111}",0.45489,0,15






American_Trends_Panel_W32


,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
prompt2_history_demo_persona,'=1463/3074,0.475927,"{1.0: 1226, 0.0: 1463, 2.0: 286, 3.0: 69, 4.0:...",0.683301,0,8
prompt2_history_persona,'=1447/3074,0.470722,"{1.0: 1243, 0.0: 1447, 2.0: 279, 3.0: 76, 4.0:...",0.689824,0,8
prompt2_persona,'=1408/3074,0.458035,"{1.0: 1390, 0.0: 1408, 3.0: 33, 2.0: 231, 4.0: 4}",0.641553,0,8
prompt2_demo_persona,'=228/499,0.456914,"{1.0: 225, 0.0: 228, 2.0: 42, 3.0: 3}",0.638554,0,1
prompt2_history_demo,'=1350/3074,0.439167,"{1.0: 1322, 0.0: 1350, 2.0: 306, 3.0: 68, 4.0:...",0.724397,0,8
prompt2_history,'=1318/3074,0.428757,"{1.0: 1332, 0.0: 1318, 2.0: 294, 3.0: 81, 4.0:...",0.761252,0,8
prompt2_demo,'=1155/3074,0.375732,"{1.0: 1421, 0.0: 1155, 2.0: 465, 3.0: 22, 4.0: 3}",0.792237,0,8






American_Trends_Panel_W34


,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
prompt2_history,'=764/1265,0.603953,"{0.0: 764, 1.0: 465, 3.0: 3, 2.0: 26}",0.418124,0,7
prompt2_history_demo,'=732/1265,0.578656,"{0.0: 732, 1.0: 494, 2.0: 28, 3.0: 4}",0.446741,0,7
prompt2_demo,'=710/1265,0.561265,"{0.0: 710, 1.0: 498, 3.0: 5, 2.0: 45}",0.479332,0,7
prompt2_history_persona,'=707/1265,0.558893,"{0.0: 707, 1.0: 527, 2.0: 22, 3.0: 2}",0.458665,0,7
prompt2_history_demo_persona,'=694/1265,0.548617,"{0.0: 694, 1.0: 536, 2.0: 27, 3.0: 1}",0.471383,0,7
prompt2_demo_persona,'=623/1265,0.49249,"{0.0: 623, 1.0: 601, 3.0: 5, 2.0: 29}",0.535771,0,7
prompt2_persona,'=607/1265,0.479842,"{0.0: 607, 1.0: 630, 3.0: 4, 2.0: 17}",0.537361,0,7
